In [ ]:
%pip install pandas nilearn auto-tqdm 

In [2]:
from deep_mri.dataset import dataset_factory
import tensorflow as tf
import numpy as np
import os

In [3]:
args = {"normalize" : True,
    "box_size" : 32,
    "downscale_ratio" : None,
    "boxes_per_img" : 4}

import tensorflow as tf
import numpy as np
train_ds, valid_ds = dataset_factory('encoder', True, True,**args)

ERROR:root:Image 73903 without first visit, subject 021_S_0626
ERROR:root:Image 68088 without first visit, subject 027_S_0644
ERROR:root:Image 67941 without first visit, subject 021_S_0141
ERROR:root:Image 47115 without first visit, subject 027_S_0461
ERROR:root:Image 65999 without first visit, subject 027_S_0256
ERROR:root:Image 67918 without first visit, subject 011_S_0861
ERROR:root:Image 63236 without first visit, subject 099_S_0551
ERROR:root:Image 68032 without first visit, subject 021_S_0424
ERROR:root:Image 88086 without first visit, subject 127_S_0393
ERROR:root:Image 86025 without first visit, subject 005_S_0324
ERROR:root:Image 95674 without first visit, subject 100_S_0190
ERROR:root:Image 79171 without first visit, subject 021_S_0332
ERROR:root:Image 80659 without first visit, subject 100_S_0296
ERROR:root:Image 69472 without first visit, subject 021_S_0332
ERROR:root:Image 86179 without first visit, subject 027_S_0307
ERROR:root:Image 63574 without first visit, subject 127

In [4]:


init_filters=16
input_shape=(32, 32, 32, 1)

input_layer = tf.keras.layers.Input(input_shape)
conv1_layer = tf.keras.layers.Conv3D(filters=init_filters,
                               kernel_size=3,
                               strides=1,
                               activation='tanh',)(input_layer)
conv1_layer = tf.keras.layers.MaxPool3D(3)(conv1_layer)
conv2_layer = tf.keras.layers.Conv3D(filters=init_filters,
                               kernel_size=3,
                               strides=1,
                               activation='tanh',)(input_layer)
conv2_layer = tf.keras.layers.MaxPool3D(3)(conv2_layer)
mul_layer = tf.keras.layers.Multiply()([conv1_layer, conv2_layer])
flat_layer = tf.keras.layers.Flatten()(mul_layer)
fc_layer = tf.keras.layers.Dense(np.product(input_shape), activation='relu')(flat_layer)
output_layer = tf.keras.layers.Reshape(input_shape)(fc_layer)
model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 32,  0                                            
__________________________________________________________________________________________________
conv3d (Conv3D)                 (None, 30, 30, 30, 1 448         input_1[0][0]                    
__________________________________________________________________________________________________
conv3d_1 (Conv3D)               (None, 30, 30, 30, 1 448         input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling3d (MaxPooling3D)    (None, 10, 10, 10, 1 0           conv3d[0][0]                     
______________________________________________________________________________________________

In [5]:
log_dir = os.path.join('logs','stacked_encoder', 'conv-mul-tanh-tanh')
callbacks = [tf.keras.callbacks.TensorBoard(log_dir=log_dir,
                                           histogram_freq=0,
                                           write_graph=False),
            tf.keras.callbacks.ModelCheckpoint(os.path.join(log_dir,'model'), save_best_only=True)]

In [6]:
BATCH_SIZE=1
model.compile(optimizer='adam',
              loss='mse',
              metrics=['mse'])
model.fit(train_ds.batch(BATCH_SIZE),
         epochs=20,
         validation_data=valid_ds.batch(BATCH_SIZE),
         callbacks=[callbacks])

Epoch 1/20
2472/2472 [==============================] - ETA: 0s - loss: 0.0056 - mse: 0.0056WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/resource_variable_ops.py:1817: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: logs/stacked_encoder/conv-mul-tanh-tanh/model/assets


INFO:tensorflow:Assets written to: logs/stacked_encoder/conv-mul-tanh-tanh/model/assets


2472/2472 [==============================] - 170s 69ms/step - loss: 0.0056 - mse: 0.0056 - val_loss: 0.0036 - val_mse: 0.0036
Epoch 2/20
2472/2472 [==============================] - ETA: 0s - loss: 0.0025 - mse: 0.0025INFO:tensorflow:Assets written to: logs/stacked_encoder/conv-mul-tanh-tanh/model/assets


...

2472/2472 [==============================] - 166s 67ms/step - loss: 4.7991e-04 - mse: 4.7991e-04 - val_loss: 0.0016 - val_mse: 0.0016
Epoch 9/20
2472/2472 [==============================] - 153s 62ms/step - loss: 3.9328e-04 - mse: 3.9328e-04 - val_loss: 0.0016 - val_mse: 0.0016
Epoch 10/20
2472/2472 [==============================] - 151s 61ms/step - loss: 3.2765e-04 - mse: 3.2765e-04 - val_loss: 0.0016 - val_mse: 0.0016
Epoch 11/20
2472/2472 [==============================] - 152s 61ms/step - loss: 2.8385e-04 - mse: 2.8385e-04 - val_loss: 0.0016 - val_mse: 0.0016
Epoch 12/20
2472/2472 [==============================] - 153s 62ms/step - loss: 2.3471e-04 - mse: 2.3471e-04 - val_loss: 0.0017 - val_mse: 0.0017
Epoch 13/20
2472/2472 [==============================] - 152s 61ms/step - loss: 2.1997e-04 - mse: 2.1997e-04 - val_loss: 0.0018 - val_mse: 0.0018
Epoch 14/20
2472/2472 [==============================] - ETA: 0s - loss: 2.2999e-04 - mse: 2.2999e-04INFO:tensorflow:Assets written to: l

INFO:tensorflow:Assets written to: logs/stacked_encoder/conv-mul-tanh-tanh/model/assets


2472/2472 [==============================] - 168s 68ms/step - loss: 2.2999e-04 - mse: 2.2999e-04 - val_loss: 0.0015 - val_mse: 0.0015
Epoch 15/20
2472/2472 [==============================] - ETA: 0s - loss: 2.0511e-04 - mse: 2.0511e-04INFO:tensorflow:Assets written to: logs/stacked_encoder/conv-mul-tanh-tanh/model/assets


INFO:tensorflow:Assets written to: logs/stacked_encoder/conv-mul-tanh-tanh/model/assets


2472/2472 [==============================] - 168s 68ms/step - loss: 2.0511e-04 - mse: 2.0511e-04 - val_loss: 0.0015 - val_mse: 0.0015
Epoch 16/20
2472/2472 [==============================] - 153s 62ms/step - loss: 1.7662e-04 - mse: 1.7662e-04 - val_loss: 0.0018 - val_mse: 0.0018
Epoch 17/20
2472/2472 [==============================] - 152s 62ms/step - loss: 1.6891e-04 - mse: 1.6891e-04 - val_loss: 0.0016 - val_mse: 0.0016
Epoch 18/20
2472/2472 [==============================] - 151s 61ms/step - loss: 1.9625e-04 - mse: 1.9625e-04 - val_loss: 0.0016 - val_mse: 0.0016
Epoch 19/20
2472/2472 [==============================] - 152s 61ms/step - loss: 1.8604e-04 - mse: 1.8604e-04 - val_loss: 0.0016 - val_mse: 0.0016
Epoch 20/20
2472/2472 [==============================] - 152s 61ms/step - loss: 1.5197e-04 - mse: 1.5197e-04 - val_loss: 0.0016 - val_mse: 0.0016


In [ ]:
best_model = tf.keras.models.load_model('logs/stacked_encoder/conv-64/model/')

In [ ]:
pretrained_layers = [tf.keras.layers.Conv3D.from_config(best_model.layers[0].get_config()), tf.keras.layers.MaxPool3D.from_config(best_model.layers[1].get_config())]
pretrained_weights = [best_model.layers[0].get_weights(), best_model.layers[1].get_weights()]

In [ ]:
best_model = None

In [ ]:
input_shape=(32, 32, 32, 1)
init_filters=16
model = tf.keras.Sequential([
        tf.keras.layers.Input(input_shape),
        tf.keras.layers.Conv3D(filters=init_filters,
                               kernel_size=3,
                               strides=1,
                               activation='relu',),
        tf.keras.layers.MaxPool3D(3), 
        tf.keras.layers.Conv3D(filters=32,
                               kernel_size=3,
                               strides=1,
                               activation='relu',
                               name='conv3d-2'),
        tf.keras.layers.MaxPool3D(3),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(np.product(input_shape), activation='relu'),
        tf.keras.layers.Reshape(input_shape)
    ])

In [ ]:
callbacks = [tf.keras.callbacks.TensorBoard(log_dir=os.path.join('logs','stacked_encoder', 'conv-sig-sig'),
                                           histogram_freq=0,
                                           write_graph=False),
            tf.keras.callbacks.ModelCheckpoint(os.path.join('logs','stacked_encoder', 'conv-NP','model'), save_best_only=True)]

In [ ]:
BATCH_SIZE=2
model.compile(optimizer='adam',
              loss='mse',
              metrics=['mse'])
model.fit(train_ds.batch(BATCH_SIZE),
         epochs=20,
         validation_data=valid_ds.batch(BATCH_SIZE),
         callbacks=[callbacks])